## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Martin 
- Apellido: Aiscar
- Legajo: 1635487

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
#Se realiza la carga del data set

hogares=pd.read_csv(r'C:\Users\marti\Data Science\tp02\TP2_airbnb_london.csv')

#Se verifica que se haya cargado correctamente

hogares.head(8367)

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285
...,...,...,...,...,...,...,...,...,...
8362,10697,Wandsworth,Private room,35,2,41,1.09,2,0
8363,10699,Haringey,Private room,35,2,21,0.69,1,338
8364,10700,Islington,Entire home/apt,83,2,122,3.18,1,186
8365,10701,Richmond upon Thames,Entire home/apt,124,2,7,0.18,1,0


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
#Se presenta la dimensiones del dataset como información adicional para futuros pasos a realizar

print('El dataset contiene '+str(hogares.shape[0])+' samples y '+str(hogares.shape[1])+' features')

El dataset contiene 51827 samples y 9 features


In [5]:
#Se elimina la variable "Unnamed:0" y se guarda el dataset en la nueva variable "london"

london=hogares.drop(hogares.columns[0],axis=1)

print('El nuevo dataset contiene '+str(london.shape[0])+' samples y '+str(london.shape[1])+' features')

El nuevo dataset contiene 51827 samples y 8 features


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [6]:
#Verificamos la cantidad de nulos dentro del dataset

cant_NaN=london.isnull().sum()
print(cant_NaN, '\r\n')

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64 



A partir de estos resultados, se puede concluir que no existen valores nulos dentro del dataset, por lo que se procede al siguiente punto sin hacer ninguna modificación adicional

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [7]:
#Se muestra el tipo de dato de cada una de las variables del dataset

london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   51827 non-null  object 
 1   room_type                       51827 non-null  object 
 2   price                           51827 non-null  int64  
 3   minimum_nights                  51827 non-null  int64  
 4   number_of_reviews               51827 non-null  int64  
 5   reviews_per_month               51827 non-null  float64
 6   calculated_host_listings_count  51827 non-null  int64  
 7   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.2+ MB


Podemos ver que tanto la variable "neighbourhood" como "room_type" son de tipo string.

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [8]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
list_vars1=["neighbourhood"]
neigh_dummies = pd.get_dummies(london[list_vars1])

Imprimir las dimensiones y primeras lineas de estas dummies

In [9]:
#Se muestra las dimensiones de las dummies de la variable neighbourhood

neigh_dummies.shape

(51827, 33)

In [10]:
#Se muestran las primeras lineas de las dummies generadas

neigh_dummies.head()

,neighbourhood_Barking and Dagenham,neighbourhood_Barnet,neighbourhood_Bexley,neighbourhood_Brent,neighbourhood_Bromley,neighbourhood_Camden,neighbourhood_City of London,neighbourhood_Croydon,neighbourhood_Ealing,neighbourhood_Enfield,...,neighbourhood_Merton,neighbourhood_Newham,neighbourhood_Redbridge,neighbourhood_Richmond upon Thames,neighbourhood_Southwark,neighbourhood_Sutton,neighbourhood_Tower Hamlets,neighbourhood_Waltham Forest,neighbourhood_Wandsworth,neighbourhood_Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
list_vars2=["room_type"]
room_dummies = pd.get_dummies(london[list_vars2])

Imprimir las dimensiones y primeras lineas de estas dummies

In [12]:
room_dummies.shape

(51827, 3)

In [13]:
room_dummies.head()

,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [14]:
#No realizo el join de las dummies generadas ya que se generara en un ColumnTransformer mas adelante

#london= london.join(room_dummies)
#london=london.join(neigh_dummies)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [15]:
#No se realiza el drop de las variables 'neighbourhood' y 'room_type' ya que se conservara para el ColumnTransformer

y = np.array(london[["price"]])
x = london.drop(['price'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4) 

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


Se realiza las siguientes lineas de codigo para poder transformar cada una de la features del dataset al momento del fitting, evitando realizar la misma transformacion para cada uno de ellas, sabiendo que existen variables numericas y categoricas. Se realiza esto ya que luego de dividir el dataset en train y test, el mismo "mezclado" aleatoriamente, perdiendo la posicion original para poder hacer el join de las dummies generadas.

In [17]:
#Variables Numéricas
numeric_features = ["minimum_nights", "number_of_reviews",'reviews_per_month','calculated_host_listings_count','availability_365']
#Variables Categóricas
variables_cat1 = ['room_type']
variables_cat2 = ['neighbourhood']

In [18]:
#StandardScaler para las variables numéricas
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
#OneHotEncoder para las variables categóricas (se generan dummies)
transformacion_cat1 = OneHotEncoder(handle_unknown="error")
transformacion_cat2 = OneHotEncoder(handle_unknown="error",sparse=False)

In [19]:
#ColumnTransformer que será ejecutado al momento del fitting
preprocessor = ColumnTransformer(
    transformers=[ 
        ("num", numeric_transformer, numeric_features),
        ("cat1", transformacion_cat1, variables_cat1),
        ("cat2", transformacion_cat2, variables_cat2),
    ]
)


In [20]:
#Se determina un Pipeline, donde se junta el ColumnTransformer con el estimador a utilizar mas adelante
svr = Pipeline(
    steps=[("preprocessor", preprocessor), ("estimador", SVR())]
)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [21]:
#Se entrena el modelo con xtrain escaldo e ytrain
x_train_scal=pd.DataFrame(preprocessor.fit_transform(xtrain))

reg=LinearRegression().fit(x_train_scal,ytrain)

x_train_scal

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,-0.850545,-0.461566,-0.634310,-0.152859,-0.945950,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.416021,-0.520318,0.495583,-0.097322,-0.327345,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.850545,-0.432189,0.280696,-0.152859,-0.945950,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.416021,-0.490942,-0.759084,-0.111206,-0.945950,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.850545,-0.520318,-0.509537,-0.152859,-0.945950,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36273,-0.850545,-0.285308,-0.156012,-0.152859,1.339870,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36274,-0.217262,-0.490942,0.946155,-0.152859,1.460573,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36275,0.416021,-0.520318,-0.807607,-0.152859,-0.908230,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36276,0.416021,0.008455,-0.523400,-0.138975,1.701980,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#Se realizan la predicciones con el dataset xtest escalado y se guarda en la variable pred para poder realizar metricas a futuro
x_test_scal=pd.DataFrame(preprocessor.fit_transform(xtest))

ypred=reg.predict(x_test_scal)
ypred

array([[138.66845703],
       [152.04101562],
       [ 53.39941406],
       ...,
       [107.91601562],
       [100.02099609],
       [153.12841797]])

In [23]:
#Se calcula el error cuadrado medio entre las etiquetas destinadas al test y las etiquetas predichas por el modelo

mean_squared_error(ytest,ypred)

1546.6561134737078

In [24]:
#Se calcula el coeficiente de correlacion del modelo

reg.score(x_train_scal,ytrain)

0.5124022218768449

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [35]:
# Hyperparametros
parameters = {'estimador__kernel':('linear','rbf'),
              'estimador__C':[1, 10, 100],
              'estimador__gamma':[0.0001,0.001, 0.01]}

In [36]:
# GridSearchCV
svreg = GridSearchCV(svr, # modelo
                     param_grid = parameters, # Hyperparametros
                     refit = True, # refit nos devuelve el modelo con los mejores parametros encontrados 
                     cv = 5, # cv indica la cantidad de folds
                     verbose = 90,
                     n_jobs=-1)


In [37]:
svreg.fit(xtrain, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['minimum_nights',
                                                                          'number_of_reviews',
                                                                          'reviews_per_month',
                                                                          'calculated_host_listings_count',
                  

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [38]:
print("The best parameters are %s with a score of %0.2f" % (svreg.best_params_, svreg.best_score_))

The best parameters are {'estimador__C': 100, 'estimador__gamma': 0.01, 'estimador__kernel': 'rbf'} with a score of 0.50


In [39]:
pd.DataFrame(preprocessor.fit_transform(xtest))

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.659529,-0.484030,-0.786379,-0.151254,-0.937559,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.221504,-0.284245,0.431739,-0.120418,1.824510,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.848516,0.315109,-0.258293,-0.105001,1.559654,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.848516,0.486354,-0.011853,-0.151254,1.446144,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.221504,-0.055919,-0.272375,-0.135836,-0.937559,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15544,0.405507,-0.341326,-0.124511,-0.151254,-0.619732,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15545,1.032518,-0.426948,-0.821585,-0.151254,0.492663,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15546,-0.848516,0.200947,-0.413198,-0.151254,1.521817,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15547,-0.848516,-0.113001,-0.427281,-0.151254,0.227807,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
ypred=svreg.predict(xtest)

In [41]:
mean_squared_error(ytest,ypred)

1576.1523804051435

In [42]:
print(svreg.score(xtest,ytest.ravel()))

0.49924834126105155


##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [27]:
# Hyperparametros
parameters = {'estimador__kernel':['rbf'],
              'estimador__C':[200,500,1000],
              'estimador__gamma':[0.001, 0.01,0.1]}

In [28]:
svreg = GridSearchCV(svr, # modelo
                     param_grid = parameters, # Hyperparametros
                     refit = True, # refit nos devuelve el modelo con los mejores parametros encontrados 
                     cv = 5, # cv indica la cantidad de folds
                     verbose = 90,
                     n_jobs=-1)

In [29]:
svreg.fit(xtrain, ytrain.ravel())

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['minimum_nights',
                                                                          'number_of_reviews',
                                                                          'reviews_per_month',
                                                                          'calculated_host_listings_count',
                  

In [30]:
print("The best parameters are %s with a score of %0.2f" % (svreg.best_params_, svreg.best_score_))

The best parameters are {'estimador__C': 200, 'estimador__gamma': 0.1, 'estimador__kernel': 'rbf'} with a score of 0.51


In [31]:
pd.DataFrame(preprocessor.fit_transform(xtest))

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.659529,-0.484030,-0.786379,-0.151254,-0.937559,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.221504,-0.284245,0.431739,-0.120418,1.824510,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.848516,0.315109,-0.258293,-0.105001,1.559654,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.848516,0.486354,-0.011853,-0.151254,1.446144,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.221504,-0.055919,-0.272375,-0.135836,-0.937559,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15544,0.405507,-0.341326,-0.124511,-0.151254,-0.619732,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15545,1.032518,-0.426948,-0.821585,-0.151254,0.492663,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15546,-0.848516,0.200947,-0.413198,-0.151254,1.521817,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15547,-0.848516,-0.113001,-0.427281,-0.151254,0.227807,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
ypred=svreg.predict(xtest)

In [33]:
mean_squared_error(ytest,ypred)

1540.1488509963513

In [34]:
print(svreg.score(xtest,ytest.ravel()))

0.5106868463802552


En contraposición con el primer gridsearch ejecutado, en este segundo podemos ver como se subieron los valores de Costo (C) y se mantuvo el mismo kernel "rbf" y valores similares de gamma seteados en el primer gridsearch. De esta manera, penalizando mucho mas el modelo por valores mas predichos, se logra obtener un score ligeramente supieror al primer caso